In [ ]:
from datasets import load_dataset

def load_wikiart_dataset():
    dataset = load_dataset("huggan/wikiart", split="train")
    return dataset
dataset = load_wikiart_dataset()
dataset.column_names

Resolving data files:   0%|          | 0/72 [00:00<?, ?it/s]

['image', 'artist', 'genre', 'style']

## Tranformar as colunas em texto